In [3]:
import os
import multiprocessing
from multiprocessing import Pool
from pathlib import Path
import uuid

import pandas as pd
from itertools import repeat
from PcapGenericParserHelper import convert_pcap, read_parse_generic

In [4]:
input_folder = '../data'
call_type_file = os.path.join(input_folder, "type.txt")
output_folder = f'../data/output/{uuid.uuid4()}'
output_folder_path = Path(output_folder)
output_csv_filename = 'generic_parser_output.csv'
pcap_folder_path = Path(input_folder)
os.makedirs(output_folder, exist_ok=True)

In [5]:
num_processors = multiprocessing.cpu_count()

In [6]:
# Fetch ALL files that end in PCAP
pcap_file_names = [ file_name for file_name in os.listdir(input_folder) if file_name.endswith("pcap") ]

In [7]:
len(pcap_file_names)

4

In [8]:
# Create raw file names of pcap files without extension ".pcap"
files=[file_name.split('.pcap')[0] for file_name in pcap_file_names]

In [9]:
num_processors = multiprocessing.cpu_count()
p = Pool(processes = num_processors)

In [10]:
# Convert PCAP to JSON
pcaps_list = p.starmap(convert_pcap, zip(repeat(pcap_folder_path), repeat(output_folder_path), files))

In [11]:
# Fetch ALL files that end in json
json_files = [ file_name for file_name in os.listdir(output_folder_path) if file_name.endswith("json") ]
len(json_files)

4

In [12]:
# Read and parse json packets, generates a nest List
call_type = open(call_type_file, "r").read()
parse_output = p.starmap(read_parse_generic, zip(repeat(output_folder_path), json_files, repeat(call_type)))

In [13]:
# Get output from original parser and concatenate both
df_out_final = pd.DataFrame(data=parse_output)
df_out_final.fillna("*", inplace=True)
df_out_final = df_out_final.set_index(df_out_final.columns[0])
df_out_final.index.names = ['pcap']

In [14]:
df_out_final

,1,2,3,4,5,6,7,8,9,10,...,100,101,102,103,104,105,106,107,108,109
pcap,,,,,,,,,,,,,,,,,,,,,
timothy.vogel@verizonwireless.com_Scheduled_PS911_v4_Eric_4TX_19-09-12_10_48_00_647_+16692481447_357330090901661_iPhone-XR_12.3_054741,UE-REGISTER-P-CSCF,P-CSCF-401-UE,UE-REGISTER-P-CSCF,P-CSCF-200-UE,-200-S-CSCF,P-CSCF-200-UE,UE-INVITE-P-CSCF,P-CSCF-100-UE,SR-Signaling-100-S-CSCF,SR-Signaling-404-S-CSCF,...,*,*,*,*,*,*,*,*,*,*
timothy.vogel@verizonwireless.com_Scheduled_PS911_v4_Eric_4TX_19-09-12_10_36_00_637_+17014801708_354386060552127_iPhone-6_Plus_11.4.1_053554,-200-,UE-INVITE-P-CSCF,P-CSCF-100-UE,SR-Signaling-100-S-CSCF,SR-Signaling-404-S-CSCF,S-CSCF-ACK-SR-Signaling,S-CSCF-INVITE-SR-Signaling,-INVITE-P-CSCF,SR-Signaling-100-S-CSCF,P-CSCF-100-,...,-ACK-P-CSCF,-200-,SR-Signaling-404-,-200-,S-CSCF-REGISTER-,-200-S-CSCF,-200-,S-CSCF-REGISTER-,-200-S-CSCF,-200-
timothy.vogel@verizonwireless.com_Scheduled_PS911_v4_Eric_4TX_19-09-12_10_06_00_612_+16692481447_357330090901661_iPhone-XR_12.3_050557,UE-INVITE-P-CSCF,P-CSCF-100-UE,SR-Signaling-100-S-CSCF,SR-Signaling-404-S-CSCF,S-CSCF-ACK-SR-Signaling,-INVITE-P-CSCF,S-CSCF-INVITE-SR-Signaling,P-CSCF-100-,SR-Signaling-100-S-CSCF,SR-Signaling-404-S-CSCF,...,-ACK-P-CSCF,-200-,-200-S-CSCF,SR-Signaling-404-,-200-,*,*,*,*,*
timothy.vogel@verizonwireless.com_Scheduled_PS911_v4_Eric_4TX_19-09-12_10_24_00_627_+17014801700_356165090145724_iPhone-XS_12.4_052415,-200-,-200-S-CSCF,-200-,SR-Signaling-404-,-200-,UE-INVITE-P-CSCF,P-CSCF-100-UE,SR-Signaling-100-S-CSCF,SR-Signaling-404-S-CSCF,S-CSCF-ACK-SR-Signaling,...,-200-,P-CSCF-404-,-ACK-P-CSCF,-200-,SR-Signaling-404-,-200-,-200-S-CSCF,-200-,*,*


In [15]:
df_out_final.to_csv(os.path.join(output_folder, output_csv_filename))
